In [2]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
df['keyword'].isnull().value_counts()

False    7552
True       61
Name: keyword, dtype: int64

In [5]:
df['location'].isnull().value_counts()

False    5080
True     2533
Name: location, dtype: int64

In [6]:
print(df['location'].value_counts(), '\n')
df.loc[df['location']== 'United States'] = 'USA'
df

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64 



,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [7]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="most_frequent")
location = np.array(df['location'])
impdf1 = imp.fit_transform(location.reshape(-1, 1))
impdf1 = pd.DataFrame(impdf1, columns = ['location'])


df['location'] = impdf1
df['keyword'] = df['keyword'].fillna('fatalities')
df

,id,keyword,location,text,target
0,1,fatalities,USA,Our Deeds are the Reason of this #earthquake M...,1
1,4,fatalities,USA,Forest fire near La Ronge Sask. Canada,1
2,5,fatalities,USA,All residents asked to 'shelter in place' are ...,1
3,6,fatalities,USA,"13,000 people receive #wildfires evacuation or...",1
4,7,fatalities,USA,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,fatalities,USA,Two giant cranes holding a bridge collapse int...,1
7609,10870,fatalities,USA,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,fatalities,USA,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,fatalities,USA,Police investigating after an e-bike collided ...,1


In [8]:
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer as PS
from nltk.tokenize import WordPunctTokenizer as WPT
import re

In [9]:
stopwords_list = stopwords.words('english')


# Clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = text.strip()
    text = ' '.join([word for word in text.split() if word not in stopwords_list])
    return text
df['text_red'] = df['text'].apply(lambda text: clean_text(text))

In [10]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ee853fd4ab6b2a0179b25c48b92aa0991ea07b463cb1778aed6f466ef18bfbf7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [11]:
from sentence_transformers import SentenceTransformer
enc_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [12]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")


In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from unidecode import unidecode
X_train = df.text.apply(unidecode).to_numpy()
X_test = test.text.apply(unidecode).to_numpy()

y_train = train.target.to_numpy()
X_train

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [15]:
X_train = enc_model.encode(X_train)
X_valid = enc_model.encode(X_valid)
X_test = enc_model.encode(X_test)

Batches:   0%|          | 0/215 [00:00<?, ?it/s]

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

In [27]:
X_train

array([[-0.04971455, -0.08053886, -0.04419315, ..., -0.41679418,
         0.2991853 ,  0.04181503],
       [-0.3826205 , -0.14494286,  0.10789994, ..., -0.44221202,
        -0.35545537,  0.03019444],
       [ 0.05669274, -0.04683261,  0.05015726, ...,  0.3014005 ,
        -0.71938294, -0.29211545],
       ...,
       [-0.88791054,  0.633504  , -0.21509637, ..., -0.57302487,
        -0.08450834,  0.10958087],
       [ 0.17209122, -0.08538444, -0.25653988, ...,  0.03025117,
         0.36303577,  0.39528307],
       [ 0.23806721, -0.26601475,  0.34578672, ...,  0.0271409 ,
        -0.38056064,  0.05868936]], dtype=float32)

In [16]:
import lightgbm as lgb

In [17]:
params = {
    'objective': 'binary'
}

In [18]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [19]:
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    num_boost_round=2000,
    callbacks=[lgb.early_stopping(10)]
)

[LightGBM] [Info] Number of positive: 2935, number of negative: 3916
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057769 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97920
[LightGBM] [Info] Number of data points in the train set: 6851, number of used features: 384
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.428405 -> initscore=-0.288363
[LightGBM] [Info] Start training from score -0.288363
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[59]	training's binary_logloss: 0.219765	valid_1's binary_logloss: 0.437311


In [20]:
y_pred1 = model.predict(X_valid)
y_pred1 = (y_pred1 > 0.5).astype(int)
print(accuracy_score(y_valid, y_pred1))

0.8202099737532809


In [21]:
test_ans = model.predict(X_test)
test_ans = (test_ans > 0.5).astype(int)
submission_df = pd.DataFrame({"id": test.id, "target": test_ans})

In [22]:
submission_df.to_csv("submission2.csv", index=False)

In [26]:
from sklearn.ensemble import RandomForestClassifier
cat = RandomForestClassifier(max_depth = 60, n_estimators = 500, random_state=42)
cat.fit(X_train,y_train)
y_pred = cat.predict(X_valid)
y_pred = (y_pred > 0.5).astype(int)
print(accuracy_score(y_valid, y_pred))

0.8149606299212598


In [24]:
y_preds = cat.predict(X_test)
y_preds = (y_preds > 0.5).astype(int)

In [25]:
submission_df = pd.DataFrame({"id": test.id, "target": y_preds})
submission_df.to_csv("submission2.csv", index=False)